In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

#CSV Output File
data_co2="annual-co-emissions-by-region.csv"
data_country = "GlobalLandTemperaturesByCountry.csv"

In [2]:
co2_df = pd.read_csv(data_co2) 
co2_df

,Entity,Code,Year,Annual CO2 emissions
0,Afghanistan,AFG,1750,0.0
1,Afghanistan,AFG,1751,0.0
2,Afghanistan,AFG,1752,0.0
3,Afghanistan,AFG,1753,0.0
4,Afghanistan,AFG,1754,0.0
...,...,...,...,...
63175,Zimbabwe,ZWE,2015,12170460.0
63176,Zimbabwe,ZWE,2016,10814761.0
63177,Zimbabwe,ZWE,2017,10246841.0
63178,Zimbabwe,ZWE,2018,11340575.0


In [3]:
clean_df=co2_df.loc[co2_df['Year'] > 1899]

In [4]:
clean_df

,Entity,Code,Year,Annual CO2 emissions
150,Afghanistan,AFG,1900,0.0
151,Afghanistan,AFG,1901,0.0
152,Afghanistan,AFG,1902,0.0
153,Afghanistan,AFG,1903,0.0
154,Afghanistan,AFG,1904,0.0
...,...,...,...,...
63175,Zimbabwe,ZWE,2015,12170460.0
63176,Zimbabwe,ZWE,2016,10814761.0
63177,Zimbabwe,ZWE,2017,10246841.0
63178,Zimbabwe,ZWE,2018,11340575.0


In [5]:
clean_df = clean_df.reset_index()
del clean_df['index']
clean_df.head

<bound method NDFrame.head of             Entity Code  Year  Annual CO2 emissions
0      Afghanistan  AFG  1900                   0.0
1      Afghanistan  AFG  1901                   0.0
2      Afghanistan  AFG  1902                   0.0
3      Afghanistan  AFG  1903                   0.0
4      Afghanistan  AFG  1904                   0.0
...            ...  ...   ...                   ...
28075     Zimbabwe  ZWE  2015            12170460.0
28076     Zimbabwe  ZWE  2016            10814761.0
28077     Zimbabwe  ZWE  2017            10246841.0
28078     Zimbabwe  ZWE  2018            11340575.0
28079     Zimbabwe  ZWE  2019            10374287.0

[28080 rows x 4 columns]>

In [6]:
bins = [1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]
group_names = ["1900-1909", "1910-1919", "1920-1929", "1930-1939", "1940-1949", "1950-1959", "1960-1969", "1970-1979", "1980-1989", "1990-1999", "2000-2009", "2010-2019"]

clean_df["Decade"] = pd.cut(clean_df["Year"], bins, labels=group_names, include_lowest=True)
clean_df

,Entity,Code,Year,Annual CO2 emissions,Decade
0,Afghanistan,AFG,1900,0.0,1900-1909
1,Afghanistan,AFG,1901,0.0,1900-1909
2,Afghanistan,AFG,1902,0.0,1900-1909
3,Afghanistan,AFG,1903,0.0,1900-1909
4,Afghanistan,AFG,1904,0.0,1900-1909
...,...,...,...,...,...
28075,Zimbabwe,ZWE,2015,12170460.0,2010-2019
28076,Zimbabwe,ZWE,2016,10814761.0,2010-2019
28077,Zimbabwe,ZWE,2017,10246841.0,2010-2019
28078,Zimbabwe,ZWE,2018,11340575.0,2010-2019


In [7]:
decade_co2 = clean_df.groupby(['Decade', 'Entity']).agg({'Annual CO2 emissions': ['mean', 'min', 'max']})
decade_co2

Annual CO2 emissions                \
                                                    mean           min   
Decade    Entity                                                         
1900-1909 Afghanistan                       0.000000e+00  0.000000e+00   
          Africa                            9.998390e+06  2.330304e+06   
          Albania                           0.000000e+00  0.000000e+00   
          Algeria                           0.000000e+00  0.000000e+00   
          Andorra                           0.000000e+00  0.000000e+00   
...                                                  ...           ...   
2010-2019 Wallis and Futuna Islands         2.487422e+04  2.198400e+04   
          World                             3.535434e+10  3.420958e+10   
          Yemen                             1.570664e+07  9.945288e+06   
          Zambia                            4.842892e+06  2.781413e+06   
          Zimbabwe                          1.063058e+07  7.658857e+06   

                                                   
                                              max  
Decade    Entity                                   
1900-1909 Afghanistan                0.000000e+00  
          Africa                     1.741133e+07  
          Albania                    0.000000e+00  
          Algeria                    0.000000e+00  
          Andorra                    0.000000e+00  
...                                           ...  
2010-2019 Wallis and Futuna Islands  2.853800e+04  
          World                      3.644139e+10  
          Yemen                      2.497675e+07  
          Zambia                     6.930094e+06  
          Zimbabwe                   1.217046e+07  

[2808 rows x 3 columns]

In [8]:
idx = pd.IndexSlice

In [9]:
main_country=decade_co2.loc[idx[:,['Brazil', 'India', 'Russia', 'United States', 'China']], :]
main_country

Annual CO2 emissions                            
                                        mean           min           max
Decade    Entity                                                        
1900-1909 Brazil                2.775313e+06  0.000000e+00  4.198944e+06
1910-1919 Brazil                3.790042e+06  2.322976e+06  6.034608e+06
1920-1929 Brazil                5.110804e+06  2.890896e+06  6.345662e+06
1930-1939 Brazil                5.295289e+06  4.168949e+06  6.485943e+06
1940-1949 Brazil                8.262202e+06  5.866699e+06  1.965907e+07
1950-1959 Brazil                3.330168e+07  2.165935e+07  4.685182e+07
1960-1969 Brazil                6.565488e+07  4.914450e+07  9.353049e+07
1970-1979 Brazil                1.507154e+08  1.023579e+08  1.871551e+08
1980-1989 Brazil                1.883839e+08  1.654269e+08  2.121136e+08
1990-1999 Brazil                2.685593e+08  2.172277e+08  3.242260e+08
2000-2009 Brazil                3.504233e+08  3.175083e+08  4.112402e+08
2010-2019 Brazil                4.777342e+08  4.299594e+08  5.238940e+08
1900-1909 India                 1.837390e+07  1.194965e+07  2.495016e+07
1910-1919 India                 3.359210e+07  2.482419e+07  4.304904e+07
1920-1929 India                 3.997993e+07  3.696578e+07  4.247764e+07
1930-1939 India                 4.299928e+07  3.553230e+07  5.240601e+07
1940-1949 India                 5.311746e+07  4.672818e+07  6.120165e+07
1950-1959 India                 8.345108e+07  6.412489e+07  1.114907e+08
1960-1969 India                 1.542900e+08  1.205832e+08  1.819556e+08
1970-1979 India                 2.392040e+08  1.922030e+08  2.920831e+08
1980-1989 India                 4.248835e+08  3.153627e+08  5.786870e+08
1990-1999 India                 7.924028e+08  6.161060e+08  9.784270e+08
2000-2009 India                 1.275682e+09  9.925292e+08  1.678479e+09
2010-2019 India                 2.248403e+09  1.766065e+09  2.616449e+09
1900-1909 Russia                5.103940e+07  4.437571e+07  6.021473e+07
1910-1919 Russia                5.283508e+07  1.901624e+07  7.534021e+07
1920-1929 Russia                4.769369e+07  2.062280e+07  9.568733e+07
1930-1939 Russia                2.013469e+08  1.149592e+08  2.976815e+08
1940-1949 Russia                2.676264e+08  1.422101e+08  4.156740e+08
1950-1959 Russia                6.629949e+08  4.504909e+08  8.858611e+08
1960-1969 Russia                1.179079e+09  9.135253e+08  1.442530e+09
1970-1979 Russia                1.848432e+09  1.530889e+09  2.132301e+09
1980-1989 Russia                2.291440e+09  2.091653e+09  2.525294e+09
1990-1999 Russia                1.693804e+09  1.458157e+09  2.395978e+09
2000-2009 Russia                1.559521e+09  1.495484e+09  1.636694e+09
2010-2019 Russia                1.649268e+09  1.618304e+09  1.691360e+09
1900-1909 United States         9.697364e+08  6.641230e+08  1.276140e+09
1910-1919 United States         1.493924e+09  1.263459e+09  1.751951e+09
1920-1929 United States         1.753823e+09  1.430738e+09  1.960553e+09
1930-1939 United States         1.556552e+09  1.256453e+09  1.871653e+09
1940-1949 United States         2.330513e+09  2.039421e+09  2.577397e+09
1950-1959 United States         2.710035e+09  2.483296e+09  2.887820e+09
1960-1969 United States         3.504613e+09  2.877370e+09  4.325501e+09
1970-1979 United States         4.650222e+09  4.351309e+09  4.894043e+09
1980-1989 United States         4.625146e+09  4.301971e+09  5.128301e+09
1990-1999 United States         5.527101e+09  5.078890e+09  5.998070e+09
2000-2009 United States         5.937367e+09  5.491036e+09  6.131893e+09
2010-2019 United States         5.408500e+09  5.253606e+09  5.565294e+09
1900-1909 China                 9.337537e+06  0.000000e+00  2.273146e+07
1910-1919 China                 1.894691e+07  9.786544e+06  2.784640e+07
1920-1929 China                 3.233252e+07  2.528893e+07  3.790792e+07
1930-1939 China                 5.617787e+07  3.858455e+07  8.604662e+07
1940-1949

In [10]:
main_country.to_csv('co2_data2.csv')

In [22]:
main_country_co2_pie = main_country.groupby(['Decade', 'Entity'])
main_country_co2_pie.head()

Annual CO2 emissions                            
                                        mean           min           max
Decade    Entity                                                        
1900-1909 Brazil                2.775313e+06  0.000000e+00  4.198944e+06
1910-1919 Brazil                3.790042e+06  2.322976e+06  6.034608e+06
1920-1929 Brazil                5.110804e+06  2.890896e+06  6.345662e+06
1930-1939 Brazil                5.295289e+06  4.168949e+06  6.485943e+06
1940-1949 Brazil                8.262202e+06  5.866699e+06  1.965907e+07
1950-1959 Brazil                3.330168e+07  2.165935e+07  4.685182e+07
1960-1969 Brazil                6.565488e+07  4.914450e+07  9.353049e+07
1970-1979 Brazil                1.507154e+08  1.023579e+08  1.871551e+08
1980-1989 Brazil                1.883839e+08  1.654269e+08  2.121136e+08
1990-1999 Brazil                2.685593e+08  2.172277e+08  3.242260e+08
2000-2009 Brazil                3.504233e+08  3.175083e+08  4.112402e+08
2010-2019 Brazil                4.777342e+08  4.299594e+08  5.238940e+08
1900-1909 India                 1.837390e+07  1.194965e+07  2.495016e+07
1910-1919 India                 3.359210e+07  2.482419e+07  4.304904e+07
1920-1929 India                 3.997993e+07  3.696578e+07  4.247764e+07
1930-1939 India                 4.299928e+07  3.553230e+07  5.240601e+07
1940-1949 India                 5.311746e+07  4.672818e+07  6.120165e+07
1950-1959 India                 8.345108e+07  6.412489e+07  1.114907e+08
1960-1969 India                 1.542900e+08  1.205832e+08  1.819556e+08
1970-1979 India                 2.392040e+08  1.922030e+08  2.920831e+08
1980-1989 India                 4.248835e+08  3.153627e+08  5.786870e+08
1990-1999 India                 7.924028e+08  6.161060e+08  9.784270e+08
2000-2009 India                 1.275682e+09  9.925292e+08  1.678479e+09
2010-2019 India                 2.248403e+09  1.766065e+09  2.616449e+09
1900-1909 Russia                5.103940e+07  4.437571e+07  6.021473e+07
1910-1919 Russia                5.283508e+07  1.901624e+07  7.534021e+07
1920-1929 Russia                4.769369e+07  2.062280e+07  9.568733e+07
1930-1939 Russia                2.013469e+08  1.149592e+08  2.976815e+08
1940-1949 Russia                2.676264e+08  1.422101e+08  4.156740e+08
1950-1959 Russia                6.629949e+08  4.504909e+08  8.858611e+08
1960-1969 Russia                1.179079e+09  9.135253e+08  1.442530e+09
1970-1979 Russia                1.848432e+09  1.530889e+09  2.132301e+09
1980-1989 Russia                2.291440e+09  2.091653e+09  2.525294e+09
1990-1999 Russia                1.693804e+09  1.458157e+09  2.395978e+09
2000-2009 Russia                1.559521e+09  1.495484e+09  1.636694e+09
2010-2019 Russia                1.649268e+09  1.618304e+09  1.691360e+09
1900-1909 United States         9.697364e+08  6.641230e+08  1.276140e+09
1910-1919 United States         1.493924e+09  1.263459e+09  1.751951e+09
1920-1929 United States         1.753823e+09  1.430738e+09  1.960553e+09
1930-1939 United States         1.556552e+09  1.256453e+09  1.871653e+09
1940-1949 United States         2.330513e+09  2.039421e+09  2.577397e+09
1950-1959 United States         2.710035e+09  2.483296e+09  2.887820e+09
1960-1969 United States         3.504613e+09  2.877370e+09  4.325501e+09
1970-1979 United States         4.650222e+09  4.351309e+09  4.894043e+09
1980-1989 United States         4.625146e+09  4.301971e+09  5.128301e+09
1990-1999 United States         5.527101e+09  5.078890e+09  5.998070e+09
2000-2009 United States         5.937367e+09  5.491036e+09  6.131893e+09
2010-2019 United States         5.408500e+09  5.253606e+09  5.565294e+09
1900-1909 China                 9.337537e+06  0.000000e+00  2.273146e+07
1910-1919 China                 1.894691e+07  9.786544e+06  2.784640e+07
1920-1929 China                 3.233252e+07  2.528893e+07  3.790792e+07
1930-1939 China                 5.617787e+07  3.858455e+07  8.604662e+07
1940-1949

In [26]:
main_country_co2_pie.set_index('Entity')

AttributeError: 'DataFrameGroupBy' object has no attribute 'set_index'